In [135]:
import json

In [136]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
template = {
    "roi": {
        "name": "Bangladesh",
        "region": False,
    },
    "dates": {
        "start": "2021-01-01",
        "end": "2022-07-01"
    },
    "data": {
        "mosaic_period": 3,
        "mosaic_method": "min",
        "spectrogram_interval": 2
    },
    "pixel": {
        "version": "0.0.11",
        "name": "spectrogram_v0.0.11_2021-07-13",
    },
    "patch": {
        "version": "weak_labels_3.0",
        "name": "weak_labels_3.0_batchnorm",
        "stride": 8,
    },
    "candidate_detect": {
        "pred_threshold": 0.6,
        "min_sigma": 5.0
    },
    "run_local": False
}

In [138]:
country_list = ['Fiji', 'Papua New Guinea', 'Solomon Islands', 'Marshall Islands', 'Micronesia', 'New Zealand', 'Vanuatu', 'Samoa', 'Palau', 'Kiribati', 'Tuvalu', 'Nauru', 'Tonga', 'Cyprus', 'Lebanon', 'Iran', 'Jordan', 'Saudi Arabia', 'Kuwait', 'Qatar', 'Bahrain', 'United Arab Emirates', 'Oman', 'Yemen', 'Nepal', 'Bhutan', 'Pakistan', 'Afghanistan', 'Armenia', 'Azerbaijan', 'Georgia']

In [139]:
for country in country_list:
    config = template.copy()
    config['roi']['name'] = country.lower().replace(' ', '_')
    #config = json.dumps(config)
    with open(f"../pipeline/{country.lower().replace(' ', '_')}_config.json", 'w') as f:
        json.dump(config, f, indent=4)
    

In [142]:
from scripts import step_1_generate_roi
failures = {}
for country in country_list:
    print(country)
    try:
        step_1_generate_roi.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        failures[country] = e
        print(f"{country} failed: {e}")
        continue

Fiji
Fiji failed: Polygon goes beyond +180deg longitude
Papua New Guinea
Approx DL Time: 2.58 hours
Number of DL Tiles to process: 1291
Wrote dlkeys to ../data/boundaries/dlkeys/papua_new_guinea_pop_0.25_dlkeys.txt
Solomon Islands
No boundaries found for Solomon_Islands
No boundaries found for solomon_islands
Solomon Islands failed: 'GetROI' object has no attribute 'region_file_name'
Marshall Islands
No boundaries found for Marshall_Islands
No boundaries found for marshall_islands
Marshall Islands failed: 'GetROI' object has no attribute 'region_file_name'
Micronesia
No boundaries found for Micronesia
No boundaries found for micronesia
Micronesia failed: 'GetROI' object has no attribute 'region_file_name'
New Zealand
Approx DL Time: 1.57 hours
Number of DL Tiles to process: 783
Wrote dlkeys to ../data/boundaries/dlkeys/new_zealand_pop_0.25_dlkeys.txt
Vanuatu
Approx DL Time: 0.30 hours
Number of DL Tiles to process: 148
Wrote dlkeys to ../data/boundaries/dlkeys/vanuatu_pop_0.25_dlkeys.t

In [145]:
from scripts import step_2_deploy
detect_failures = {}
for country in [c for c in country_list if c not in failures.keys() and != 'Samoa']:
    print(country)
    try:
        step_2_deploy.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        detect_failures[country] = e
        print(f"{country} failed: {e}")
        continue

Samoa
Existing product found.
Deleting db97e4b985ef52fa21d7e83dd688047fada1a0f2:earthrise:samoa_patch_weak_labels_3.0_2021-01-01_2022-07-01_stride_8_patches
Read 44 dlkeys from ../data/boundaries/dlkeys/samoa_pop_0.25_dlkeys.txt
Estimated cost for run: $0.71
Got patch product earthrise:samoa_v0.0.11_2021-01-01_2022-07-01
Model spectrogram_v0.0.11_2021-07-13 found in DLStorage.
Model weak_labels_3.0_batchnorm found in DLStorage.
Creating product earthrise:samoa_patch_weak_labels_3.0_2021-01-01_2022-07-01_stride_8_patches


100%|██████████| 44/44 [00:10<00:00,  4.05it/s]


In [115]:
from scripts import step_3_detect_candidates
candidate_failures = {}
for country in [c for c in country_list if c not in detect_failures.keys() and c not in failures.keys()]:
    print(country)
    try:
        step_3_detect_candidates.main(config_path=f"../pipeline/{country.lower().replace(' ', '_')}_config.json")
    except KeyboardInterrupt:
        print("Keyboard interrupt")
        break
    except Exception as e:
        candidate_failures[country] = e
        print(f"{country} failed: {e}")
        continue

{'Fiji': descarteslabs.common.dltile.exceptions.InvalidShapeError('Polygon goes beyond +180deg longitude'),
 'Solomon Islands': AttributeError("'GetROI' object has no attribute 'region_file_name'"),
 'Marshall Islands': AttributeError("'GetROI' object has no attribute 'region_file_name'"),
 'Micronesia': AttributeError("'GetROI' object has no attribute 'region_file_name'"),
 'Tuvalu': pandas.errors.EmptyDataError('No columns to parse from file')}